# Master Workflow

## This notebook includes all steps necessary from data download and forcing file preparation to ensemble generation and setup.

---


In [27]:
from aislens import dataprep, geospatial, utils, generator, plot
from aislens.config import config
import xarray as xr

KeyError: 0

## 1. Introduction

This section provides an overview of the workflow, including the objectives and the main steps involved in the process.

---

### Raw data files required:

| Data | Description | File Name |
| --- | --- | --- |
| Mean Melt, Draft | Satellite observations of basal melt | `ANT_G1920V01_IceShelfMeltDraft.nc` | 
| TF - Trend | ISMIP6 ocean TF file(s) | `UKESM1-0-LL_SSP585_thermal_forcing_8km_x_60m.nc` |
| TF Param Coeffs | gamma0 and deltaT fields for ISMIP6 TF | `coeff_gamma0_DeltaT_quadratic_local_median.nc` |
| Melt Variability | MPAS-Ocean SORRMv2.1 simulation run | `Regridded_SORRMv2.1.ISMF.FULL.nc` |
| Ice Shelf Masks | Shapefiles for ice shelves | `iceShelves.geojson` |

* `ANT_G1920V01_IceShelfMeltDraft.nc`: This file includes `melt` and `draft` variables.
    * The melt rate is averaged across time to obtain an initial condition field of freshwater flux to be used in MALI simulations. 
    * The melt and draft are used together to derive draft dependence parameters, `alpha0` and `alpha1`, which are used to derive online values of the melt field as the MALI simulation progresses with ice sheet retreat.

* `UKESM1-0-LL_SSP585_thermal_forcing_8km_x_60m.nc`: 






In [6]:
paolo23_satobs = xr.open_dataset(Config.FILE_PAOLO23_SATOBS)
mpaso_model = xr.open_dataset(Config.FILE_MPASO_MODEL)


In [7]:
paolo23_satobs

<xarray.Dataset>
Dimensions:  (x: 2916, y: 2916, time: 104)
Coordinates:
  * x        (x) float64 -2.798e+06 -2.796e+06 ... 2.796e+06 2.798e+06
  * y        (y) float64 2.798e+06 2.796e+06 2.795e+06 ... -2.796e+06 -2.798e+06
  * time     (time) datetime64[ns] 1992-03-17T03:04:19.200002304 ... 2017-12-...
Data variables:
    mapping  int64 ...
    melt     (time, y, x) float32 ...
    draft    (time, y, x) float32 ...
Attributes:
    units:          m of ice per year
    standard_name:  ice shelf basal melt rate
    density:        917.0
    density_units:  kg m^-3

In [33]:


def prepare_satellite_observations():
    # Load satellite observation dataset
    satobs = xr.open_dataset(config.FILE_PAOLO23_SATOBS)
    
    # Detrend the data along the time dimension
    satobs_detrended = detrend_dim(satobs[config.SATOBS_FLUX_VAR], dim=config.TIME_DIM, deg=1)
    
    # Deseasonalize the data
    satobs_deseasonalized = deseasonalize(satobs_detrended)
    
    # Save the prepared data
    satobs_deseasonalized.to_netcdf(config.FILE_PAOLO23_SATOBS_PREPARED)

def prepare_model_simulation():
    # Load model simulation dataset
    model = xr.open_dataset(config.FILE_MPASO_MODEL)
    
    # Subset the dataset to the desired time range
    model_subset = model.sel({config.TIME_DIM: slice(config.START_YEAR, config.END_YEAR)})
    
    # Detrend, deseasonalize, and dedraft the data
    model_detrended = detrend_dim(model_subset[config.SORRM_FLUX_VAR], dim=config.TIME_DIM, deg=1)
    model_deseasonalized = deseasonalize(model_detrended)
    model_dedrafted = dedraft(model_deseasonalized, model_subset[config.SORRM_DRAFT_VAR])
    
    # Save the processed components
    model_deseasonalized.to_netcdf(config.FILE_SEASONALITY)
    model_dedrafted.to_netcdf(config.FILE_VARIABILITY)

if __name__ == "__main__":
    prepare_satellite_observations()
    prepare_model_simulation()

NameError: name 'config' is not defined

NameError: name 'xr' is not defined

## 2. Data Download

Instructions and code to download the required datasets for the workflow.

---

The data is stored in the `data/` subdirectory of this repository. All main files we require are downloaded to the `data/external/` subdirectory, and these files are processed as necessary and final files are saved in the `data/processed/` subdirectory.

Data directory tree:
```bash
├── data
│   ├── external    # Download all external data files here.
│   ├── interim     # Interim files during preprocessing and visualization steps are saved here. 
│   ├── processed   # Prepared and processed data files that are ready to use are saved here.
│   ├── raw         # Alternative to external, as required. These may be files in a different grid.
│   └── tmp         # Temporary interim files that can be deleted after runs are saved here.
```

## 3. Data Preprocessing

Steps to clean, preprocess, and organize the downloaded data for further analysis.

---



## 4. Forcing File Preparation

Preparation of forcing files required for the modeling or analysis workflow. This section prepares the original model-based forcing file.

---



## 5. Ensemble Generation

Procedures to generate realizations of forcing files for members of the ice sheet model ensemble for uncertainty quantification or scenario analysis.

---



## 6. Ensemble Setup

Configuration and setup of the generated ensembles for subsequent simulations or analyses.

---



## 7. Results and Visualization

Presentation and visualization of the results obtained from the workflow.

---

## 8. Conclusion

Summary of the workflow, key findings, and next steps.